In [10]:
# Set number of cores for parallel computation
KERNEL = 10


import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
import sys,os
import random
import copy

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2

from methods.utils import interpret_results, resultsHandler

from pathlib import Path
import multiprocessing as mp

from utils.eval import find_best_matches, make_known_groups, find_best_matching_biclusters


def make_ref_groups(subtypes, annotation,exprs):
    # prepared a dict of subtype classifications {"class1":{"subt1":[],"subt2":[]},"class2":{"subtA":[],"subtB":[]}}
    all_samples = set(exprs.columns.values)
    pam50 = make_known_groups(subtypes, exprs,target_col = "PAM50",verbose=False)
    lum = {}
    lum["Luminal"] = pam50["LumA"].union(pam50["LumB"])
    scmod2 = make_known_groups(subtypes, exprs,target_col = 'SCMOD2',verbose=False)
    claudin = {} 
    claudin["Claudin-low"] = set(subtypes.loc[subtypes['claudin_low']==1,:].index.values).intersection(all_samples)
    
    ihc = {}
    for x in ["IHC_HER2","IHC_ER","IHC_PR"]:
        ihc[x] = set(annotation.loc[annotation[x]=="Positive",:].index.values)
    ihc["IHC_TNBC"] = set(annotation.loc[annotation["IHC_TNBC"]==1,:].index.values)
    
    known_groups = {"PAM50":pam50,"Luminal":lum,"Claudin-low":claudin,"SCMOD2":scmod2,"IHC":ihc}
    
    weights = {}
    N = 0
    for classification in known_groups.keys():
        for group in known_groups[classification].keys():
            n = len(known_groups[classification][group])
            weights[group] = n
            N+=n
    for group in weights.keys():
        weights[group] = weights[group]/N
    
    return known_groups, weights

def calculate_perfromance(results, known_groups, weights, all_samples):
    # finds best matches for each subtype, calcuates J per subtype and overall performance
    N = len(all_samples)
    best_matches = []
    
    for classification in known_groups.keys():
        bm = find_best_matches(results,known_groups[classification],all_samples,FDR=0.05,verbose = False)
        best_matches.append(bm)
            
    best_matches = pd.concat(best_matches, axis=0)
    best_matches = best_matches["J"].to_dict()
    overall_performance = 0
    for group in best_matches.keys():
        overall_performance += best_matches[group]*weights[group]
    best_matches["overall_performance"] = overall_performance
    return best_matches

def compare_gene_clusters(tcga_result,metabric_result, N):
    # N - total number of genes
    # finds best matched TCGA -> METABRIC and METABRIC -> TCGA
    # calculates % of matched clusterst, number of genes in matched cluster, 
    # and the average J index for best matches 
    bm = find_best_matching_biclusters(tcga_result,metabric_result, N)
    bm = bm.dropna()
    bm2 = find_best_matching_biclusters(metabric_result, tcga_result, N)
    bm2 = bm2.dropna()
    
    bm = bm.loc[bm["n_shared"]>1,:].sort_values(by="n_shared",ascending = False)
    bm2 = bm2.loc[bm2["n_shared"]>1,:].sort_values(by="n_shared",ascending = False)
    
    
    clust_similarity = {}
    # number of biclusters 
    clust_similarity["n_1"] = tcga_result.shape[0]
    clust_similarity["n_2"] = metabric_result.shape[0]
    #print("% matched biclusters:",bm.shape[0]/tcga_result.shape[0],bm2.shape[0]/metabric_result.shape[0])
    clust_similarity["percent_matched_1"] = bm.shape[0]/tcga_result.shape[0]
    clust_similarity["percent_matched_2"] = bm2.shape[0]/metabric_result.shape[0]
    #print("n matched genes:",bm.loc[:,"n_shared"].sum(),bm2.loc[:,"n_shared"].sum())
    clust_similarity["n_shared_genes_1"] = bm.loc[:,"n_shared"].sum()
    clust_similarity["n_shared_genes_2"] = bm2.loc[:,"n_shared"].sum()
    #print("avg. J:",bm.loc[:,"J"].mean(),bm2.loc[:,"J"].mean())
    clust_similarity["avg_bm_J_1"] = bm.loc[:,"J"].mean()
    clust_similarity["avg_bm_J_2"] = bm2.loc[:,"J"].mean()
    
    
    return clust_similarity, bm, bm2




file_metabric_annotation = '/home/hartung/data/preprocessed_v6/METABRIC_1904.annotation_v6.tsv'
file_metabric_expression = '/home/hartung/data/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_z_v6.tsv'
file_metabric_subtypes = '/home/hartung/data/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv'
file_tcga_annotation = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv'
file_tcga_expression = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv'
file_tcga_subtypes = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv'
file_gene_mapping = '/home/hartung/data/preprocessed_v6/gene_id_mapping.tsv'

out_dir= '/home/hartung/data/preprocessed_v6/results'

basename_t = "TCGA"
basename_m = "METABRIC" 


m_subtypes = pd.read_csv(file_metabric_subtypes,sep = "\t",index_col=0)
m_annotation = pd.read_csv(file_metabric_annotation,sep = "\t",index_col=0)

t_subtypes = pd.read_csv(file_tcga_subtypes,sep = "\t",index_col=0)
t_annotation = pd.read_csv(file_tcga_annotation,sep = "\t",index_col=0)


exprs_t= pd.read_csv(file_tcga_expression,sep = "\t",index_col=0)
exprs_t[exprs_t>3] = 3
exprs_t[exprs_t<-3] = -3

exprs_m= pd.read_csv(file_metabric_expression,sep = "\t",index_col=0)
exprs_m[exprs_m>3] = 3
exprs_m[exprs_m<-3] = -3


METHODS = [moCluster, MOFA2] # [NMF, sparse_PCA, moCluster, MOFA2]
for METHOD in METHODS:
    method_name = METHOD.__name__.split('.')[-1]
    
    #### Preparation
    # METABRIC
    file_path_m = file_metabric_expression
    output_path_m = os.path.join(out_dir, basename_m, method_name)
    ground_truth_file_m = file_metabric_annotation
    combinations_m = METHOD.generate_arg_list(file_path_m, output_path_m, ground_truth_file_m, cluster_range=[10, 100])
    # TCGA
    file_path_t = file_tcga_expression
    output_path_t = os.path.join(out_dir, basename_t, method_name)
    ground_truth_file_t = file_tcga_annotation
    combinations_t = METHOD.generate_arg_list(file_path_t, output_path_t, ground_truth_file_t, cluster_range=[10, 100])

    
    #### Compute in parallel
    # Option to compute the results in parallel, methods will store results
    # Follow up with executing the 'Run' below to read existing results and evaluate
    if KERNEL > 1:
        with mp.Pool(KERNEL) as pool:
            pool.map(METHOD.run_real, combinations_m + combinations_t)

            
    #### Run
    # Methods will compute results or read existing results
    # sanity check
    assert len(combinations_m) == len(combinations_t)
    subt_t = []
    subt_m = []
    clustering_similarities = []
    for comb_m, comb_t in zip(combinations_m, combinations_t):
        result_m, runtime_m = METHOD.run_real(comb_m)
        result_t, runtime_t = METHOD.run_real(comb_t)
        
        try:
            m_best_matches = match_known_subtypes(result_m, m_subtypes, m_annotation,exprs_m)
            m_best_matches = m_best_matches["J"].to_dict()
        except ZeroDivisionError:
            m_best_matches = {}
        subt_m.append(m_best_matches)

        try:
            t_best_matches = match_known_subtypes(result_t, t_subtypes, t_annotation,exprs_t)
            t_best_matches = t_best_matches["J"].to_dict()
        except ZeroDivisionError:
            t_best_matches = {}
        subt_t.append(t_best_matches)
        
    # save results
    pd.DataFrame.from_records(subt_m).to_csv(os.path.join(out_dir, basename_m, method_name, 'best_cluster_matches.csv'))    
    pd.DataFrame.from_records(subt_t).to_csv(os.path.join(out_dir, basename_t, method_name, 'best_cluster_matches.csv'))


Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=6/n_cluster=100/random_state=1...Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=10/random_state=2...Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=100/random_state=0...Returning existing results:Returning existing results:Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=6/n_cluster=100/random_state=0...Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=100/random_state=1...

  
/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=10/random_state=0/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=10/random_state=1



Returning existing results:Returning existing results:Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=6/n_cluster=1

Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table


      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 2 ...
calculating component 2 ...
calculating component 2 ...
calculating component 2 ...
Saved /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=100/random_state=2.
Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=2/n_cluster=100/random_state=2...
Saved /home/hartung/data/preprocessed_v6/results

Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table


      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 1 ...
calculating component 4 ...
calculating component 4 ...
calculating component 4 ...
calculating component 4 ...
calculating component 4 ...
calculating component 2 ...
calculating component 2 ...
calculating component 2 ...
calculating component 2 ...
calculating component 2 ...
calculating component 5 ...
calculating component 5 ...
calculating component 5 ...
calculating component 5 ...
calculating component 5 ...
Saved /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=2/n_cluster=100/random_state=2.
Running /home/hartung/

...

Warning message:
In run_mofa(MOFAobject, outfile) :
  The latest mofapy2 version is 0.6.4, you are using 0.6.6. Please upgrade with 'pip install mofapy2'


Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=1/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=1/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=2/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=2/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=3/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=3/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=4/n_cluster=10/random_state=0
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=4/n_cluster=10/random_state=0
Returnin

Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=4/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=4/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=5/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=5/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=6/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=6/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=7/n_cluster=100/random_state=1
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=7/n_cluster=100/random_state=1


Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=7/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=7/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=8/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=8/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=9/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=9/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=10/n_cluster=10/random_state=3
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=10/n_cluster=10/random_state=3
Return

Returning existing results: /home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/n_factors=10/n_cluster=100/random_state=4
Returning existing results: /home/hartung/data/preprocessed_v6/results/TCGA/MOFA2/n_factors=10/n_cluster=100/random_state=4


In [50]:
import pandas as pd


df = pd.read_csv('/home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/best_cluster_matches.csv')

In [51]:
df

,Unnamed: 0,LumB,LumA,Her2,Normal,Basal,Luminal,Claudin-low,ER-/HER2-,ER+/HER2- High Prolif,ER+/HER2- Low Prolif,HER2+,IHC_TNBC,IHC_PR,IHC_ER,IHC_HER2
0,0,0.143860,0.160780,0.048148,0.055901,0.138805,0.191534,0.154286,0.198830,0.193069,0.000000,0.000000,0.0,0.000000,0.188729,0.000000
1,1,0.173673,0.164921,0.236074,0.055814,0.534296,0.324723,0.155844,0.452128,0.181601,0.192463,0.204545,0.0,0.209259,0.382022,0.211640
2,2,0.237172,0.274924,0.225904,0.218543,0.477663,0.321033,0.325153,0.387909,0.263092,0.291304,0.151420,0.0,0.216834,0.364045,0.153179
3,3,0.273632,0.244378,0.170306,0.153409,0.151930,0.488930,0.335227,0.496583,0.267643,0.254649,0.112786,0.0,0.297207,0.188769,0.132494
4,4,0.290557,0.258595,0.185841,0.084375,0.440433,0.271218,0.092219,0.343669,0.263024,0.264151,0.118012,0.0,0.186425,0.312360,0.132184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.036316,0.048414,0.056000,0.169492,0.098361,0.047970,0.127907,0.068120,0.037090,0.046512,0.052885,0.0,0.028571,0.056180,0.045455
96,96,0.035202,0.058333,0.067460,0.119403,0.101626,0.049815,0.186047,0.068120,0.038849,0.057364,0.070755,0.0,0.029050,0.053933,0.069672
97,97,0.035202,0.052893,0.084337,0.154930,0.085366,0.042435,0.151163,0.060109,0.041847,0.057187,0.065421,0.0,0.028515,0.051685,0.056225
98,98,0.040155,0.053512,0.087302,0.206897,0.093878,0.044280,0.137931,0.063014,0.051799,0.049612,0.069124,0.0,0.034483,0.049438,0.064000
